<a href="https://colab.research.google.com/github/beanapologist/Quantum-Duality-Theory/blob/main/QIO_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts

# Define constants
PHI = 1.618033988749
PI = 3.14159265359
LAMBDA = 0.867
GAMMA = 0.4497
BETA = 0.310
ETA = 0.52
EMERGENCE = 0.580002
CMB_COUPLING = 3.67e-5
DYNAMIC_LR_SCALE = 0.9
STABILIZATION_DELTA = 1e-6
LAMBDA_REG = 1e-4
NOISE_STD = 1e-6

# Define the model
model = nn.Linear(1, 1)  # Example model

# Example data
x = torch.randn(100, 1)
y = 3 * x + 2 + torch.randn(100, 1) * 0.1  # Target with some noise

# Define optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.05)
criterion = nn.MSELoss()

# Warm restart scheduler
scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2, eta_min=1e-7)

# Training loop parameters
converged = False
prev_loss = float('inf')  # For monitoring loss improvement
stabilization_epochs = 0  # Counter for stabilization
steady_lr = 1e-5  # Minimum learning rate after convergence

# Training loop
for epoch in range(100):  # Train for up to 100 epochs

    # Zero gradients
    optimizer.zero_grad()

    # Forward pass
    predictions = model(x)

    # Compute original loss
    original_loss = criterion(predictions, y)

    # Add regularization to push predictions closer to true values
    zeroing_loss = LAMBDA_REG * torch.sum((predictions - y) ** 2)
    loss = original_loss + zeroing_loss

    # Backpropagation
    loss.backward()

    # Inject gradient noise post-convergence to escape local minima
    if converged:
        for p in model.parameters():
            if p.grad is not None:
                noise = torch.randn_like(p.grad) * NOISE_STD
                p.grad.add_(noise)

    # Apply gradient clipping to stabilize optimization
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=ETA if converged else PHI)

    # Optimization step
    optimizer.step()

    # Monitor convergence
    if loss.item() < 1.0 and not converged:
        print(f"Convergence detected at Epoch {epoch + 1}. Switching to post-convergence strategies.")
        for param_group in optimizer.param_groups:
            param_group['lr'] = steady_lr  # Set steady learning rate
        converged = True

    # Post-convergence learning rate adjustments
    if converged:
        if prev_loss > loss.item():  # If loss improves
            # Dynamically scale the learning rate
            dynamic_lr = optimizer.param_groups[0]["lr"] * DYNAMIC_LR_SCALE * (1 - (loss.item() / prev_loss))
            for param_group in optimizer.param_groups:
                param_group["lr"] = max(dynamic_lr, CMB_COUPLING)  # Prevent LR from dropping too low
        else:  # Use warm restart to escape local minima
            scheduler.step()

    # Check for loss stabilization and terminate if stabilized
    if abs(prev_loss - loss.item()) < STABILIZATION_DELTA and converged:
        print(f"Loss has stabilized at {loss.item():.6f}. Terminating training.")
        break

    # Scheduler adjustment for non-converged phase
    if not converged:
        scheduler.step()

    # Update previous loss for monitoring
    prev_loss = loss.item()
    print(f"Epoch {epoch + 1}, Loss: {loss.item():.6f}, LR: {optimizer.param_groups[0]['lr']:.6f}")